# TP1: Conversion of Sampling Frequency and STFT

*By Daniel Jorge Deutsch, Kevin Kuhl and Brayam Santiago Velandia (25/09/2020)*

In [ ]:
import os
import struct
import sys
import wave
from copy import deepcopy
from math import ceil

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyaudio
from scipy import signal as sig
from scipy.io import wavfile

In [ ]:
class Signal:

    def __init__(self, name, signal=None, freq=None, file=None):

        if not (file or not signal is None):
            raise Exception("You must at least pass a signal or a .wav file")
        if file and not signal is None:
            raise Exception("You can only pass a signal or a .wav file (not both)")

        self.name = name 

        if not signal is None:
            self.signal = np.asarray(signal)
            self.freq = int(freq) if freq else freq
        if file:
            self.file = file
            self.freq, self.signal = wavfile.read(self.file)
            self.freq = int(self.freq)
            self.signal = np.asarray(self.signal)
            
    
    #------------------------------#
    #--- OPERATIONS ---------------#
    #------------------------------#

    def convolve(self, name, signal):
        freq = signal.freq if signal.freq else self.freq
        conv = np.convolve(self.signal, signal.signal)
        return self.__class__(name, signal=conv, freq=freq)


    #------------------------------#
    #--- SAMPLING -----------------#
    #------------------------------#

    def remove_samples(self, name, M, start=0):
        res = self.signal[start::M]
        freq = self.freq/M if self.freq else self.freq
        return self.__class__(name=name, signal=res, freq=self.freq/M)


    def insert_zeros(self, name, L):
        res = np.insert(self.signal, range(1, len(self.signal)+1)[::L-1], 0)
        freq = self.freq*L if self.freq else self.freq
        return self.__class__(name=name, signal=res, freq=self.freq*L)


    #------------------------------#
    #--- PLOTS --------------------#
    #------------------------------#

    def time_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):
        
        # Time axis
        t = np.linspace(-0.5, 0.5, len(self.signal)) ################### ISSO TA ERRADO

        # Plot
        plt.figure(figsize=figsize)
        plt.plot(t, self.signal)
        plt.xlim(t[0], t[-1])
        plt.xlabel("Time [s]")
        plt.ylabel("Amplitude")
        plt.title(f"Time plot of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(x=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_time.png", dpi=300, bbox_inches="tight")
        plt.show()

    
    # Should use when you have the filter's coefficients
    def freqz_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):

        freq, mag = sig.freqz(self.signal)

        # Magnitude
        mag = 20*np.log10(np.abs(mag))
        mag[np.abs(mag-np.mean(mag)) > 2*np.std(mag)] = np.nan     # Remove outliers for the plot

        # Frequency   
        freq = freq/(2*np.pi)
        
        # Plot
        plt.figure(figsize=figsize)
        plt.plot(freq, mag)
        plt.xlim(freq[0], freq[-1])
        plt.ylabel("Magnitude [(]dB]")
        plt.xlabel("Frequency [(rad/sample)/2π]")
        plt.title(f"Frequency response of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(y=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_freqz.png", dpi=300, bbox_inches="tight")
        plt.show() 

    
    # Should use when you have filter's impulse response
    def fft_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):      
        
        # Magnitude
        len_fft = 4096                                           # Length of the transformed axis of the output
        mag = np.fft.fft(self.signal, len_fft)                   # Obtains the magnitude of the signal
        mag = np.fft.fftshift(mag)                               # Shifts the zero-frequency component to the center of the spectrum
        mag = 20*np.log10(np.abs(mag))                           # Applies log to the result
        mag[np.abs(mag-np.mean(mag)) > 2*np.std(mag)] = np.nan   # Remove outliers for the plot

        # Frequency
        freq = np.linspace(-0.5, 0.5, len(mag))

        # Plot
        plt.figure(figsize=figsize)
        plt.plot(freq, mag)
        plt.xlim(freq[0], freq[-1])
        plt.ylabel("Magnitude [dB]")
        plt.xlabel("Frequency [(rad/sample)/2π]")
        plt.title(f"Frequency response of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(y=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_fft.png", dpi=300, bbox_inches="tight")
        plt.show() 

    
    #------------------------------#
    #--- LISTEN -------------------#
    #------------------------------#
    
    def listen(self):
        
        # Saves the .wav
        if not self.freq:
            raise Exception("the sample freq (sample/sec) must be provided")
        wavfile.write(f"./outputs/sounds/{self.name}.wav", self.freq, np.asarray(self.signal, dtype=np.int16))

        # Uses pyaudio to play the signal
        chunk = 1024
        pa = pyaudio.PyAudio()
        audio = wave.open(f"./outputs/sounds/{self.name}.wav", "rb")
        stream = pa.open(
            format = pa.get_format_from_width(audio.getsampwidth()),
            channels = audio.getnchannels(),
            rate = audio.getframerate(),
            output = True
        )
        data = audio.readframes(chunk)
        while data:
            stream.write(data)
            data = audio.readframes(chunk)
        stream.stop_stream()
        stream.close()
        pa.terminate()

In [ ]:
class Filter:

    def __init__(self, name, signal):
        self.name = name
        self.signal = signal

    
    def apply(self, name, signal):
        conv = np.convolve(self.signal, signal.signal)
        return Signal(name, signal=conv, freq=signal.freq)

    
    def time_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):
        
        # Time axis
        t = np.linspace(-0.5, 0.5, len(self.signal)) ################### ISSO TA ERRADO

        # Plot
        plt.figure(figsize=figsize)
        plt.plot(t, self.signal)
        plt.xlim(t[0], t[-1])
        plt.xlabel("Time [s]")
        plt.ylabel("Amplitude")
        plt.title(f"Time plot of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(x=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_time.png", dpi=300, bbox_inches="tight")
        plt.show()

    
    # Should use when you have the filter's coefficients
    def freqz_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):

        freq, mag = sig.freqz(self.signal)

        # Magnitude
        mag = 20*np.log10(np.abs(mag))
        mag[np.abs(mag-np.mean(mag)) > 2*np.std(mag)] = np.nan     # Remove outliers for the plot

        # Frequency   
        freq = freq/(2*np.pi)
        
        # Plot
        plt.figure(figsize=figsize)
        plt.plot(freq, mag)
        plt.xlim(freq[0], freq[-1])
        plt.ylabel("Magnitude [dB]")
        plt.xlabel("Frequency [(rad/sample)/2π]")
        plt.title(f"Frequency response of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(y=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_freqz.png", dpi=300, bbox_inches="tight")
        plt.show() 

    
    # Should use when you have filter's impulse response
    def fft_plot(self, xhighlights=None, yhighlights=None, figsize=(15, 4), save=False):      
        
        # Magnitude
        len_fft = 4096                                           # Length of the transformed axis of the output
        mag = np.fft.fft(self.signal, len_fft)                   # Obtains the magnitude of the signal
        mag = np.fft.fftshift(mag)                               # Shifts the zero-frequency component to the center of the spectrum
        mag = 20*np.log10(np.abs(mag))                           # Applies log to the result
        mag[np.abs(mag-np.mean(mag)) > 2*np.std(mag)] = np.nan   # Remove outliers for the plot

        # Frequency
        freq = np.linspace(-0.5, 0.5, len(mag))

        # Plot
        plt.figure(figsize=figsize)
        plt.plot(freq, mag)
        plt.xlim(freq[0], freq[-1])
        plt.ylabel("Magnitude [dB]")
        plt.xlabel("Frequency [(rad/sample)/2π)]")
        plt.title(f"Frequency response of {self.name}")
        if xhighlights:
            for xhighlight in xhighlights:
                plt.axvline(x=xhighlight, color="red")
        if yhighlights:
            for yhighlight in yhighlights:
                plt.hline(y=yhighlight, color="red")
        if save:
            plt.savefig(f"./outputs/figures/{self.name}_fft.png", dpi=300, bbox_inches="tight")
        plt.show()   

## 2.1

## 2.2

In [ ]:
import time
# Obtains the input signal x from its file
x = Signal(name="x", file="./inputs/caravan_48khz.wav")

# Defines the over and under sampling constants
L = 2
M = 3

# Obtains the over sampled signal w using L=2 (Fw=48*2=96kHz)
w = x.insert_zeros("w", L)

# Defines the Remez filter params
numtaps = 100                       # An even number (can't be too big)
trans_width = 1/13                  # ?
cutoff = min( 1/(2*L), 1/(2*M) )    # Cutoff frequency (should be 1/(2*M) = 1/6)

# Obtains the Remez filter
h = Filter(name="h", signal=sig.remez(numtaps, [0, cutoff, cutoff+trans_width, 1/2], [L, 0]))

start1 = time.perf_counter()
# Calculates the convolution v=w*h
v = h.apply(name="v", signal=w)

# Undersample the convolution result to obtain the output of the system using M=3 (Fy=96/3=32kHz)
y = v.remove_samples("y", M)
end1 = time.perf_counter()


## 2.3

![title](img/Proof.png)

## 2.4

<img src="img/01.png" width="552" height="892" />
<img src="img/02.png" width="552" height="892" />

In [ ]:

R0 = Filter("R0", h.signal[::2])
R1 = Filter("R0", h.signal[1::2])
R00 = Filter("R00", R0.signal[:-1:3])
R01 = Filter("R01", R0.signal[1::3])
R02 = Filter("R02", np.append(R0.signal[2::3],[0]))
R10 = Filter("R10", R1.signal[:-1:3])
R11 = Filter("R11", R1.signal[1::3])
R12 = Filter("R12", np.append(R1.signal[2::3], [0]))

start2 = time.perf_counter()

w0 = Signal("w0", np.append(x.signal[1:], [0]), x.freq)
w1 = Signal("w1", np.append([0],w0.signal[:-1]), x.freq)
w2 = Signal("w2", np.append([0],w1.signal[:-1]), x.freq)
w3 = Signal("w3", x.signal[:], x.freq)
w4 = Signal("w4", np.append([0],w3.signal[:-1]), x.freq)
w5 = Signal("w5", np.append([0],w4.signal[:-1]), x.freq)

v0 = w0.remove_samples("v0", 3)
v1 = w1.remove_samples("v1", 3)
v2 = w2.remove_samples("v2", 3)
v3 = w3.remove_samples("v3", 3)
v4 = w4.remove_samples("v4", 3)
v5 = w5.remove_samples("v5", 3)

u0 = R00.apply("u0", v0)
u1 = R01.apply("u1", v1)
u2 = R02.apply("u2", v2)
u3 = R10.apply("u3", v3)
u4 = R11.apply("u4", v4)
u5 = R12.apply("u5", v5)

k1 = Signal("k1", u0.signal + u1.signal + u2.signal, freq=x.freq/M)
k2 = Signal("k2", u3.signal + u4.signal + u5.signal, freq=x.freq/M)

p1 = k1.insert_zeros("p1", 2)
p2 = k2.insert_zeros("p2", 2)

q1 = np.append([0],p1.signal[:-1])

y = Signal("y", q1 + p2.signal, freq=x.freq*L/M)
end2 = time.perf_counter()
y.listen()

## 2.5

In [ ]:
print("Time period using Direct Implementation: {}".format(end1 - start1))
print("Time period using Polyphase Implementation: {}".format(end2 - start2))

# STFT
## 3.1a

In [ ]:
# Defines a Hanning window of length 50
Nw = 50                                 
w = Signal("w", np.hanning(Nw))

# Lower and upper limits of the main lobe (so width = 4/Nw)
xhighlights = [-2/Nw, 2/Nw]

# Plots the DTF of w
w.fft_plot(xhighlights=xhighlights)

In [ ]:
# Defines a Hanning window of length 31
Nw = 31
w = Signal("w", signal=np.hanning(Nw))

# Defines the variables k, R and M as requested
k = 3     # Index of the frequency channel
R = 1     # Hop size (increment on analysis times)
M = 32    # TDF order

# Number of tdf's to calculate
Nt = int((len(x.signal)-Nw)/R)

# Calculates Xtil
Xtil = np.zeros((M, Nt), dtype=complex)
for u in np.arange(0, Nt).reshape(-1):
    term = np.multiply(x.signal[np.arange(int(u*R+1), int(u*R+1+Nw))], w.signal)
    Xtil[:, u] = np.fft.fft(term, M)

# Listen to the real part of xk (with k=3)
rexk = np.real(Xtil[k, :])
rexk = Signal("rexk", signal=rexk, freq=x.freq)
rexk.listen()

In [ ]:
N = x.shape[0] # % longueur du signal
Nw = 512
w = np.hanning(Nw) # définition de la fenetre d'analyse
ws = w.copy; # définition de la fenêtre de synthèse
R = Nw/4 # incrément sur les temps d'analyse, appelé hop size, t_a=uR
M = 512 # ordre de la tfd
L = M/2+1
affich = 1 ; # pour affichage du spectrogramme, 0 pour
             # pour faire analyse/modif/synthèse sans affichage
             # note: cf. spectrogram sous Matlab
Nt = np.rint((N - Nw) / R) # calcul du nombre de tfd à calculer
Nt = Nt.astype(int)
y = np.zeros((N,1)) # signal de synthèse

if affich:
    Xtilde = np.zeros((M,Nt),dtype=complex)



for u in np.arange(0,Nt).reshape(-1): # boucle sur les trames
    deb = u * R + 1 # début de trame
    fin = deb + Nw # fin de trame
    tx = np.multiply(x.signal[np.arange(deb.astype(int),fin.astype(int))],w) # calcul de la trame 
    X = np.fft.fft(tx,M) # tfd à l'instant b
    if affich:
        Xtilde[:,u] = X
    # opérations de transformation (sur la partie \nu > 0)
    # ....
    Y = X.copy
    # fin des opérations de transformation
    # resynthèse
    # overlap add

def extents(f):
  delta = f[1] - f[0]
  return [f[0] - delta/2, f[-1] + delta/2]


if affich:
    plt.imshow(20*np.log10(np.abs(Xtilde[np.arange(0,L),:])), aspect='auto',interpolation='none',
               origin='lower', extent=[0, Nt * R / Fs + Nw / 2, 0, Fs/2])



def ola(w = None,hop = None,Nb = 10): 
# function output = ola(w,hop,Nb)
# realise l'addition-recouvrement de la fenetre w,
# avec un décalage hop et un nombre Nb de fenetres.
# par defaut Nb = 10;
    
    w = w[:, np.newaxis]
    N = len(w)
    output = np.zeros(((Nb - 1) * hop + N,1)) # réserve l'espace memoire
    
    for k in np.arange(0,Nb).reshape(-1):
        deb = k* hop
        fin = deb + N
        output[np.arange(deb,fin)] = output[np.arange(deb,fin)] + w # OLA
    
    return output
    

## 3.1b
The STFT referred to as low-pass convention is defined in discrete time as:

$$W_x(\lambda,b)=\sum_{n\in \mathbb Z} x(n)w(n-b)e^{-j2\pi\lambda n}$$

In this equation we consider the input signal $x(n)$ and the analysis window in discrete time $w(n-b)$ (where $b$ is a discret and quantized variable) to obtain the filter-bank interpretation. The spectrum of the signal $x$  is first rotated along unit circle in the $z$ plane in that way we shift $\lambda$ frecuency down to 0. Then, we can see the input signal as:

$$x(n)\overset{\Delta}{=} x(n)e^{-j2\pi\lambda n}$$

Additionally, the signal is convolved with the window to get $W_x(b)$:

$$W_x(b)=\sum_{n\in \mathbb Z} x(n)w(n-b)= (x \ast w)(b)$$

At this point, we consider de window $w$ as a low pass filter impulse response. The output $W_x$ is a time-domain signal with a centered componend $e^{-j2\pi\lambda n}$ removed by frecuency shifting. Since the analysis of the STFT is symetric and the lenght of the impulse response is odd, then we conlude that is a Type I filter, in the case of an even impulse response we define a Type II filter.

## 3.1c

Starting by the given expression:

$$\tilde{X}(\lambda,b)=\sum_{n\in \mathbb Z} x(n+b)w(n-b)e^{-j2\pi\lambda n}$$

Doing m = n+b:

$$\tilde{X}(\lambda,b)=\sum_{m\in \mathbb Z} x(m)w(m-b)e^{-j2\pi\lambda (m-b)}$$
$$\tilde{X}(\lambda,b)=\sum_{m\in \mathbb Z} x(m)w(m-b)e^{-j2\pi\lambda m}e^{j2\pi\lambda b}$$
$$\tilde{X}(\lambda,b)=e^{j2\pi\lambda b} \sum_{m\in \mathbb Z} x(m)w(m-b)e^{-j2\pi\lambda m}$$
$$\tilde{X}(\lambda,b)=e^{j2\pi\lambda b} W_x(\lambda,b)$$